In [31]:
import numpy as np
import csv

np.random.seed(1000)

In [32]:
# 하이퍼 파라미터 정의 (학습률, 정규분초에 대한 평균 및 표준 편차)
RND_MEAN = 0
RND_STD = 0.003

LEARNING_RATE = 0.001

In [33]:
# 메인 함수 정의 : 신경망 모델 생성 및 전체 과정 일괄 처리
# report : 주기
def main_exec(epoch_count = 10, mb_size = 10, report = 1, train_rate = 0.8):
    load_dataset()
    init_model()
    train_and_test(epoch_count, mb_size, report, train_rate)

In [34]:
# 데이터 셋 로드 및 비선형 정보에 대해 원-핫 벡터로 표현 과정
def load_dataset():
    with open('./data/abalone.csv') as csvfile:
        csvreader = csv.reader(csvfile)
        next(csvreader, None) # next() : 열 정보에 관한 행을 스킵
        rows = []
        for row in csvreader:
            rows.append(row)
    
    global data, input_cnt, output_cnt # 전역 변수 설정
    input_cnt, output_cnt = 10, 1 # 입출력 벡터 크기 설정
    data = np.zeros([len(rows), input_cnt + output_cnt]) # 원-핫 벡터 표현을 위한 초기화 과정

    # 원-핫 벡터 표현 과정
    for n,row in enumerate(rows):
        if row[0] == 'I' : data[n,0] = 1
        if row[0] == 'F' : data[n,1] = 1
        if row[0] == 'M' : data[n,2] = 1
        data[n,3:] = row[1:]
    

In [35]:
# 모델을 위한 파라미터 초기화 과정
def init_model():
    global weight, bias, input_cnt, output_cnt # 전역 변수 설정
    weight = np.random.normal(RND_MEAN, RND_STD, [input_cnt, output_cnt]) # 가중치 : 랜덤하게 초기화 -> 매번 다른 실험결과 + local minimum 피할 가능성 높임
    bias = np.zeros([output_cnt]) # 편향 : 모두 0으로 초기화

In [36]:
'''
이후의 과정은 학습 및 평가에 대한 과정이다.
편의상 필요한 각 내부 기능에 대해 미리 설명한 후 코드를 진행한다.

train_and_test() : 학습 및 평가 기능에 대한 총괄 함수
arrange_data() : 전체 데이터 셋을 무작위로 섞은 후, 1 에포크 당 미니배치 스텝 수 계산
get_test_data() : 테스트 데이터 셋을 분리 후, 독립 변수과 종속 변수 분할
get_train_data() : 미니배치 사이즈 만큼 처리한 후, 훈련 데이터에 대한 독립 변수와 종속 변수 분할
run_train() : 훈련 -> 순전파, 역전파를 수행, loss acc 반환
run_test() : 최종 모델에 대한 정확도 출력

'''

'\n이후의 과정은 학습 및 평가에 대한 과정이다.\n편의상 필요한 각 내부 기능에 대해 미리 설명한 후 코드를 진행한다.\n\ntrain_and_test() : 학습 및 평가 기능에 대한 총괄 함수\narrange_data() : 전체 데이터 셋을 무작위로 섞은 후, 1 에포크 당 미니배치 스텝 수 계산\nget_test_data() : 테스트 데이터 셋을 분리 후, 독립 변수과 종속 변수 분할\nget_train_data() : 미니배치 사이즈 만큼 처리한 후, 훈련 데이터에 대한 독립 변수와 종속 변수 분할\nrun_train() : 훈련 -> 순전파, 역전파를 수행, loss acc 반환\nrun_test() : 최종 모델에 대한 정확도 출력\n\n'

In [37]:
# 학습 및 평가
def train_and_test(epoch_count, mb_size, report, train_rate):
    step_count = arrange_data(mb_size, train_rate)
    test_x, test_y = get_test_data()

    for epoch in range(epoch_count):
        losses, accs = [], []
        for n in range(step_count):
            train_x, train_y= get_train_data(mb_size, n)
            loss, acc = run_train(train_x,train_y)
            losses.append(loss)
            accs.append(acc)
        
        if report > 0 and (epoch + 1) % report == 0:
            acc = run_test(test_x, test_y)
            print("Epoch {} : Train - loss = {:5.3f}, accuracy = {:5.3f} / Test = {:5.3f}".\
                format(epoch+1, np.mean(losses), np.mean(accs), acc))
    
    final_acc = run_test(test_x,test_y)
    print('\n Result : final accuracy = {:5.3f}'.format(final_acc))

In [38]:
# 이후의 과정은 train_and_test()의 하위 기능에 대한 함수를 정의한다.

In [39]:
def arrange_data(mb_size, train_rate):
    global data, shuffle_map, test_begin_index
    shuffle_map = np.arange(data.shape[0])
    np.random.shuffle(shuffle_map)

    step_count = int(data.shape[0] * train_rate) // mb_size

    test_begin_index = step_count * mb_size # 일반적으로 80% : 학습, 20% : 테스트이기에 이 사이의 경계선 idx
    return step_count

In [40]:
def get_test_data():
    global data,shuffle_map, test_begin_index, output_cnt
    test_data = data[shuffle_map[test_begin_index:]]

    return test_data[:,:-output_cnt], test_data[:,-output_cnt:]

In [41]:
def get_train_data(mb_size, nth):
    global data, shuffle_map, test_begin_index, output_cnt
    if nth == 0:
        np.random.shuffle(shuffle_map[:test_begin_index]) # 에포크마다 셔플

    train_data = data[shuffle_map[mb_size * nth : mb_size*(nth+1)]]
    
    return train_data[:,:-output_cnt], train_data[:,-output_cnt:]


In [42]:
# 학습 수행 과정 : 순전파를 통해 loss값 계산 -> 정확도 계산 -> 기울기 계산
def run_train(x,y):
    output, aux_nn = forward_neuralnet(x)
    loss, aux_pp = forward_postproc(output, y)
    accuracy = eval_accuracy(output, y)

    G_loss = 1.0
    G_output = backprop_postproc(G_loss,aux_pp)
    backprop_neuralnet(G_output,aux_nn)

    return loss, accuracy

In [43]:
#
def run_test(x, y):
    output, _ = forward_neuralnet(x)
    accuracy = eval_accuracy(output, y)
    return accuracy

In [44]:
# 손실함수 중 mse를 구하는 과정
def forward_postproc(output, y):
    diff = output - y

    square = np.square(diff)

    loss = np.mean(square)

    return loss, diff

In [53]:
#
def backprop_postproc(G_loss, diff):
    shape = diff.shape
    G_loss = 1

    g_loss_square = np.ones(shape) / np.prod(shape)
    g_square_diff = 2 * diff
    g_diff_output = 1

    G_square = g_loss_square * G_loss
    G_diff = g_square_diff * G_square
    G_output = g_diff_output * G_diff

    return G_output


In [46]:
#
def forward_neuralnet(x):
    global weight, bias
    output = np.matmul(x, weight) + bias
    return output, x


In [55]:
#
def backprop_neuralnet(G_output, x):
    global weight, bias
    g_output_w = x.transpose()

    G_w = np.matmul(g_output_w, G_output)
    G_b = np.sum(G_output, axis=0)

    weight -= LEARNING_RATE * G_w
    bias -= LEARNING_RATE * G_b



In [48]:
#
def eval_accuracy(output, y):
    mdiff = np.mean(np.abs((output - y) / y))

    return 1 - mdiff


In [56]:
main_exec()

Epoch 1 : Train - loss = 33.605, accuracy = 0.561 / Test = 0.810
Epoch 2 : Train - loss = 8.023, accuracy = 0.819 / Test = 0.815
Epoch 3 : Train - loss = 7.332, accuracy = 0.813 / Test = 0.809
Epoch 4 : Train - loss = 7.225, accuracy = 0.810 / Test = 0.808
Epoch 5 : Train - loss = 7.144, accuracy = 0.809 / Test = 0.810
Epoch 6 : Train - loss = 7.076, accuracy = 0.811 / Test = 0.807
Epoch 7 : Train - loss = 7.020, accuracy = 0.810 / Test = 0.808
Epoch 8 : Train - loss = 6.968, accuracy = 0.811 / Test = 0.807
Epoch 9 : Train - loss = 6.925, accuracy = 0.810 / Test = 0.809
Epoch 10 : Train - loss = 6.885, accuracy = 0.812 / Test = 0.807

 Result : final accuracy = 0.807
